In [61]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lightgbm import LGBMRegressor as LGBR
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [26]:
def create_df(journey):
    comunio = pd.read_csv(f'../data/pruebas/only_comunio_stats_J{journey}.csv')
    clas = pd.read_excel(f'../data/classification_J_{journey}.xlsx', sheet_name=f'classification_J_{journey}', index_col='Unnamed: 0')
    cal = pd.read_csv('../data/pruebas/Season_21-22.csv')
    
    teams_dict = {'Athletic Club': 'Athletic Club' ,
                 'CA Osasuna':'Osasuna',
                 'Club Atlético de Madrid':'Atlético Madrid',
                 'Cádiz CF':'Cádiz',
                 'Deportivo Alavés':'Alavés',
                 'Elche CF':'Elche',
                 'FC Barcelona':'Barcelona',
                 'Getafe CF':'Getafe',
                 'Granada CF':'Granada',
                 'Levante UD':'Levante',
                 'RC Celta de Vigo':'Celta Vigo',
                 'RCD Espanyol de Barcelona':'Espanyol',
                 'RCD Mallorca':'Mallorca',
                 'Rayo Vallecano de Madrid':'Rayo Vallecano',
                 'Real Betis Balompié':'Betis',
                 'Real Madrid CF':'Real Madrid',
                 'Real Sociedad de Fútbol':'Real Sociedad',
                 'Sevilla FC':'Sevilla',
                 'Valencia CF':'Valencia',
                 'Villarreal CF':'Villarreal'}
    
    new_team_name = []
    for team in comunio.Team:
  
        for k,v in teams_dict.items():
       
            if k == team:
           
                new_team_name.append(v)
    
    comunio['Team'] = new_team_name
    
    points_per_team = comunio.groupby('Team').sum().reset_index()
    points_per_team= points_per_team.rename(columns={'Total_Points': 'Squad_Points',
                                                       'Points_Average': 'Squad_Average_Points',
                                                       'Avg_last_5_games': 'Squad_Avg_last_5_Games',
                                                       'Value': 'Value_Squad',
                                                       f'J_{journey -4}': f'Squad_Points_J_{journey -4}',
                                                       f'J_{journey -3}': f'Squad_Points_J_{journey -3}',
                                                       f'J_{journey -2}': f'Squad_Points_J_{journey -2}',
                                                       f'J_{journey -1}': f'Squad_Points_J_{journey -1}',
                                                       f'J_{journey}': f'Squad_Points_J_{journey}',})
    
    points_per_team = points_per_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_1 = comunio.merge(points_per_team, how='left', left_on='Team', right_on='Team')
    
    matches_J = cal.loc[cal['Journey'] == (journey+1)]
    matches_J
    
    vs = []
    for team in df_1.Team:

        for index in matches_J.index:
           
            if team == matches_J.loc[index].Home:
            
                vs.append(matches_J.loc[index].Away)
                
            if team == matches_J.loc[index].Away:
                
                vs.append(matches_J.loc[index].Home)
    
    df_1['vs'] = vs
    
    points_per_vs_team = comunio.groupby('Team').sum().reset_index()
    points_per_vs_team= points_per_vs_team.rename(columns={'Total_Points': 'Vs_Squad_Points',
                                                           'Team':'Vs_Team',
                                                           'Points_Average': 'Vs_Squad_Average_Points',
                                                           'Avg_last_5_games': 'Vs_Squad_Avg_last_5_Games',
                                                           'Value':'Vs_Value_Squad',
                                                           f'J_{journey -4}': f'Vs_Squad_Points_J_{journey -4}',
                                                           f'J_{journey -3}': f'Vs_Squad_Points_J_{journey -3}',
                                                           f'J_{journey -2}': f'Vs_Squad_Points_J_{journey -2}',
                                                           f'J_{journey -1}': f'Vs_Squad_Points_J_{journey -1}',
                                                           f'J_{journey}': f'Vs_Squad_Points_J_{journey}',})
    
    points_per_vs_team = points_per_vs_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_2 = df_1.merge(points_per_vs_team, how='left', left_on='vs', right_on='Vs_Team')
    
    team_clas = []

    for team in comunio.Team:
        for index in clas.index:
            
            if team in clas.loc[index].Team:
                
                team_clas.append(clas.loc[index].Position)
    
    team_clas_vs = []

    for team in df_2.Vs_Team:
        for index in clas.index:
           
            if team in clas.loc[index].Team:
                
                team_clas_vs.append(clas.loc[index].Position)
    
    df_2['Team_clas'] = team_clas
    df_2['Vs_Team_clas'] = team_clas_vs
    
    
    return df_2.drop('Team_id', axis=1)

In [3]:
def preprocess_data(train,target,journey):
    j_28_target = target[['Player',f'J_{journey}']]
    j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_{journey}':'Target'})
    df= train.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
    df = df.dropna()
    X = df.drop(['Target'], axis=1)._get_numeric_data()
    y = df.Target
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    
    return X, y, X_train, X_test, y_train, y_test 

In [27]:
def preprocess_data(train,target,journey):
    j_28_target = target[['Player',f'J_{journey}']]
    j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_{journey}':'Target'})
    df= train.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
    df = df.dropna()
    X = df.drop(['Target'], axis=1)._get_numeric_data()
    y = df.Target
    
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    
    X_train_s = x_scaler.fit_transform(X_train)
    X_test_s = x_scaler.transform(X_test)
    
    y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1))
    y_test_s = y_scaler.transform(y_test.values.reshape(-1,1))
                    
    return X, y, X_train, X_test, y_train, y_test ,X_train_s, X_test_s, y_train_s, y_test_s

In [28]:
j_30 = create_df(30)
j_31 = create_df(31)

In [29]:
X, y, X_train, X_test, y_train, y_test ,X_train_s, X_test_s, y_train_s, y_test_s = preprocess_data(j_30,j_31,31)

In [30]:
rfr = RandomForestRegressor()
xgbr = XGBRegressor()
gb = GradientBoostingRegressor()
models = (rfr,xgbr,gb)

In [31]:
for model in models:
    name = str(model)[:13]
    if 'Sequential' in name:
        
        model.fit(X_train_s,
                   y_train_s,
                   epochs=50,
                   validation_data=0.15)
        pred = model.predict(X_test_s)
        pred2 = y_scaler.inverse_transform(pred)
        mse = mean_squared_error(pred2,y_test, squared=False)
        r2 = r2_score(pred2, y_test)
        
        pd.DataFrame(model.history).plot(figsize=(8,5))
        plt.grid()
        plt.show()
        
        
    if 'MLP' in name:
        
        model.fit(X_train_s,y_train_s)
        model.score(X_train_s, y_train_s)
        pred = model.predict(X_test_s)
        pred_test = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred_test,y_test, squared=False)
        
        
    else:
        '''model.fit(X_train,y_train)
        model.score(X_train, y_train)'''
        model.fit(X_train_s,y_train)
        model.score(X_train_s, y_train)
        pred = model.predict(X_test_s)
        #pred2 = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred,y_test, squared=False)
        r2 = r2_score(pred, y_test)
    
    print (f'mse del modelo {name} = {mse}')

mse del modelo RandomForestR = 2.877298718705909
mse del modelo XGBRegressor( = 2.994284008011537
mse del modelo GradientBoost = 2.8883303376744203


In [32]:
rfr.fit(X,y)
xgbr.fit(X,y)
gb.fit(X,y)

new_x = j_31._get_numeric_data()
new_x

,Total_Points,Points_Average,Value,On_start_%,J_27,J_28,J_29,J_30,J_31,Avg_last_5_games,Squad_Points,Squad_Average_Points,Value_Squad,Squad_Points_J_27,Squad_Points_J_28,Squad_Points_J_29,Squad_Points_J_30,Squad_Points_J_31,Squad_Avg_last_5_Games,Vs_Squad_Points,Vs_Squad_Average_Points,Vs_Value_Squad,Vs_Squad_Points_J_27,Vs_Squad_Points_J_28,Vs_Squad_Points_J_29,Vs_Squad_Points_J_30,Vs_Squad_Points_J_31,Vs_Squad_Avg_last_5_Games,Team_clas,Vs_Team_clas
0,122,4.5,3560000,87,2,5,2,2,5,3.2,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
1,20,5.0,390000,13,0,0,0,0,0,0.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
2,0,0.0,180000,0,0,0,0,0,0,0.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
3,115,5.5,5560000,65,7,0,0,0,0,1.4,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
4,103,5.4,3390000,58,2,4,6,2,3,3.4,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
5,82,3.6,1440000,68,0,0,0,0,0,0.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
6,81,4.8,1750000,45,3,2,2,3,3,2.6,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
7,79,4.6,2720000,55,5,4,4,0,3,3.2,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
8,77,3.2,1200000,71,0,3,0,0,6,1.8,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
9,45,6.4,4450000,16,5,4,13,8,0,6.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12


In [39]:
pred_rfr = rfr.predict(X)
pred_xgb = xgbr.predict(X)
pred_gb = gb.predict(X)
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds = round(preds)
preds
X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(preds, y, random_state=42)

In [41]:
X_train_S.shape, X_test_S.shape, y_train_S.shape, y_test_S.shape

((431, 5), (144, 5), (431,), (144,))

In [42]:
lgbr= LGBR()

lgbr.fit(X_train_S._get_numeric_data(), y_train_S._get_numeric_data())

train_score = lgbr.score(X_train_S._get_numeric_data(), y_train_S)
test_score = lgbr.score(X_test_S._get_numeric_data(), y_test_S)

'train R2 ', train_score, ' test R2 ', test_score

('train R2 ', 0.9739200730668521, ' test R2 ', 0.9901361652582392)

In [43]:
lgbr.fit(preds._get_numeric_data(), y)

LGBMRegressor()

In [46]:
pred_lgbr = lgbr.predict(preds._get_numeric_data())


In [47]:
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds['lgbm'] = pred_lgbr
preds = round(preds)
preds

,Player,Position,rfr,xgb,gb,lgbm
0,Unai Simón,PT,4.0,5.0,4.0,5.0
1,Agirrezabala,PT,0.0,-0.0,0.0,-0.0
2,Ezkieta,PT,0.0,0.0,0.0,-0.0
3,Iñigo Martínez,DF,1.0,-0.0,2.0,-0.0
4,Daniel Vivian,DF,3.0,3.0,3.0,3.0
5,Lekue,DF,1.0,0.0,1.0,-0.0
6,De Marcos,DF,3.0,3.0,2.0,3.0
7,Yeray Álvarez,DF,3.0,3.0,3.0,3.0
8,Balenziaga,DF,4.0,6.0,3.0,6.0
9,Yuri Berchiche,DF,1.0,-0.0,1.0,-0.0


In [48]:
pred_rfr = rfr.predict(new_x)
pred_xgb = xgbr.predict(new_x)
pred_gb = gb.predict(new_x)

In [50]:
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds = round(preds)
pred_lgbr = lgbr.predict(preds._get_numeric_data())
preds['lgbm'] = pred_lgbr
preds = round(preds)
preds

,Player,Position,rfr,xgb,gb,lgbm
0,Unai Simón,PT,5.0,4.0,4.0,4.0
1,Agirrezabala,PT,0.0,1.0,0.0,1.0
2,Ezkieta,PT,0.0,0.0,-0.0,-0.0
3,Iñigo Martínez,DF,3.0,2.0,2.0,2.0
4,Daniel Vivian,DF,4.0,2.0,3.0,2.0
5,Lekue,DF,1.0,-0.0,1.0,-0.0
6,De Marcos,DF,3.0,1.0,3.0,1.0
7,Yeray Álvarez,DF,3.0,4.0,4.0,4.0
8,Balenziaga,DF,3.0,4.0,4.0,4.0
9,Yuri Berchiche,DF,4.0,2.0,1.0,2.0


In [51]:
squad = ['Courtois', 'Bono',
         'Balliu', 'Fran García','Ronald Araujo', 'Alderete', 'Manu Sánchez', 'Giménez',
         'Yéremi',  'Gavi',  'Modric',  'Brais', 'Papu Gómez', 'Dani Rodríguez', 'Pere Milla', 'Cervi', 'Berenguer',
         'Vinícius Júnior', 'Aubameyang', 'Sancet', 'João Félix', 'Morales']
df_squad_kike = pd.DataFrame()
for player in squad:
    #print (player)
    for p in preds.Player:
        if player in p:
            #print('encontrado')
            add_player = pd.DataFrame(preds.loc[preds.Player==p])
            #print(add_player)
            df_squad_kike = df_squad_kike.append(add_player)
print(len(squad),len(df_squad_kike))

df_squad_kike.at[514,'Position'] = 'MD'
round(df_squad_kike)

22 22


,Player,Position,rfr,xgb,gb,lgbm
199,Courtois,PT,5.0,5.0,6.0,5.0
545,Bono,PT,3.0,1.0,2.0,1.0
233,Balliu,DF,6.0,6.0,6.0,6.0
234,Fran García,DF,4.0,5.0,5.0,5.0
109,Ronald Araujo,DF,4.0,5.0,4.0,5.0
410,Omar Alderete,DF,5.0,3.0,4.0,3.0
53,Manu Sánchez,DF,3.0,3.0,2.0,3.0
27,José Giménez,DF,3.0,4.0,6.0,4.0
393,Yéremi Pino,MD,4.0,4.0,4.0,4.0
120,Gavi,MD,4.0,3.0,4.0,3.0


In [14]:
def once_ideal_xbgr(data):
    gk = data.loc[data['Position']=='PT'].sort_values('xgb', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('xgb', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('xgb', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('xgb', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [15]:
xgb_eleven = round(once_ideal_xbgr(df_squad_kike))
xgb_eleven

,Player,Position,rfr,xgb,gb
199,Courtois,PT,5.0,5.0,6.0
233,Balliu,DF,6.0,6.0,6.0
234,Fran García,DF,4.0,5.0,5.0
109,Ronald Araujo,DF,5.0,5.0,4.0
528,Brais Méndez,MD,3.0,5.0,3.0
393,Yéremi Pino,MD,4.0,4.0,4.0
530,Franco Cervi,MD,3.0,4.0,3.0
14,Álex Berenguer,MD,4.0,4.0,5.0
221,Vinícius Júnior,DL,6.0,4.0,7.0
129,Aubameyang,DL,7.0,3.0,5.0


In [16]:
def once_ideal_rf(data):
    gk = data.loc[data['Position']=='PT'].sort_values('rfr', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('rfr', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('rfr', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('rfr', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [17]:
def once_ideal_gb(data):
    gk = data.loc[data['Position']=='PT'].sort_values('gb', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('gb', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('gb', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('gb', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [18]:
rf_eleven = round(once_ideal_rf(df_squad_kike))
rf_eleven

,Player,Position,rfr,xgb,gb
199,Courtois,PT,5.0,5.0,6.0
233,Balliu,DF,6.0,6.0,6.0
109,Ronald Araujo,DF,5.0,5.0,4.0
410,Omar Alderete,DF,5.0,3.0,4.0
393,Yéremi Pino,MD,4.0,4.0,4.0
120,Gavi,MD,4.0,3.0,4.0
297,Dani Rodríguez,MD,4.0,3.0,5.0
14,Álex Berenguer,MD,4.0,4.0,5.0
129,Aubameyang,DL,7.0,3.0,5.0
221,Vinícius Júnior,DL,6.0,4.0,7.0


In [19]:
gb_eleven = round(once_ideal_gb(df_squad_kike))
gb_eleven

,Player,Position,rfr,xgb,gb
199,Courtois,PT,5.0,5.0,6.0
233,Balliu,DF,6.0,6.0,6.0
27,José Giménez,DF,3.0,4.0,6.0
234,Fran García,DF,4.0,5.0,5.0
297,Dani Rodríguez,MD,4.0,3.0,5.0
14,Álex Berenguer,MD,4.0,4.0,5.0
393,Yéremi Pino,MD,4.0,4.0,4.0
120,Gavi,MD,4.0,3.0,4.0
221,Vinícius Júnior,DL,6.0,4.0,7.0
129,Aubameyang,DL,7.0,3.0,5.0


In [52]:
def once_ideal_lgbm(data):
    gk = data.loc[data['Position']=='PT'].sort_values('lgbm', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('lgbm', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('lgbm', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('lgbm', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [53]:
lgbm_eleven = round(once_ideal_lgbm(df_squad_kike))
lgbm_eleven

,Player,Position,rfr,xgb,gb,lgbm
199,Courtois,PT,5.0,5.0,6.0,5.0
233,Balliu,DF,6.0,6.0,6.0,6.0
234,Fran García,DF,4.0,5.0,5.0,5.0
109,Ronald Araujo,DF,4.0,5.0,4.0,5.0
528,Brais Méndez,MD,4.0,5.0,3.0,5.0
393,Yéremi Pino,MD,4.0,4.0,4.0,4.0
530,Franco Cervi,MD,3.0,4.0,3.0,4.0
14,Álex Berenguer,MD,5.0,4.0,5.0,4.0
221,Vinícius Júnior,DL,6.0,4.0,7.0,4.0
129,Aubameyang,DL,6.0,3.0,5.0,3.0


In [66]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either e

In [71]:
def grid(modelo, param, cv=5): # por fuerza bruta
    
    grid=GridSearchCV(modelo, 
                      param, 
                      cv=cv,    # cross-validation
                      iid=True, # independiente e identicamente distribuidos
                      return_train_score=True,
                      n_jobs=-1,
                      verbose = 3)
    
    grid.fit(X_train, y_train)
    
    print('Acierto test: {:.2f}'.format(grid.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(grid.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(grid.best_params_))
    print('Mejor Acierto cv: {:.2f}'.format(grid.best_score_))
    
    return grid.best_estimator_.fit(X_train, y_train)

In [58]:
help(gb)

Help on GradientBoostingRegressor in module sklearn.ensemble._gb object:

class GradientBoostingRegressor(sklearn.base.RegressorMixin, BaseGradientBoosting)
 |  GradientBoostingRegressor(*, loss='ls', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort='deprecated', validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for regression.
 |  
 |  GB builds an additive model in a forward stage-wise fashion;
 |  it allows for the optimization of arbitrary differentiable loss functions.
 |  In each stage a regression tree is fit on the negative gradient of the
 |  given loss function.
 |  
 |  Read more in the :ref:`User Guide <gradient_boosting>`.
 |  
 |  Parameters
 

In [73]:
params_rfr = {'max_leaf_nodes': [25, 30, 35], 'n_estimators': [100, 150, 200,300], 'max_depth':[5,10,20]}
params_xgbr = {'n_estimators': [100,150,200], 'learning_rate':[0.01, 0.05, 0.1], 'gamma': [0.1,0.2], 'verbose':[1]}
params_gb = {'loss':['ls','huber'],'learning_rate':[0.01, 0.05, 0.1], 'gamma': [0.1,0.2]}

In [68]:
rf_grid = grid(rfr, params_rfr)
rf_grid

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    8.3s finished
/home/kike/anaconda3/envs/Entorno_Virtual_BT_Data_Science/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


Acierto test: 0.20
Acierto train: 0.62
Mejores parametros: {'max_depth': 5, 'max_leaf_nodes': 30, 'n_estimators': 100, 'n_jobs': -1}
Mejor Acierto cv: 0.21


RandomForestRegressor(max_depth=5, max_leaf_nodes=30, n_jobs=-1)

In [69]:
rf_grid.predict(new_x)

array([4.47462527e+00, 9.73797331e-02, 4.02972830e-02, 2.63373924e+00,
       2.98273507e+00, 1.28083082e+00, 2.89846686e+00, 2.85865450e+00,
       2.96471942e+00, 3.56709409e+00, 1.29505460e+00, 1.89701638e+00,
       4.02972830e-02, 4.48540353e+00, 3.73696155e+00, 3.14141876e+00,
       2.72323163e+00, 1.37647149e+00, 3.00828625e+00, 1.74176237e+00,
       7.19356266e-01, 3.17280638e+00, 2.93821350e+00, 3.80192114e+00,
       1.89920536e+00, 3.97660137e+00, 4.28532952e-02, 2.87480058e+00,
       2.60394996e+00, 1.54271157e+00, 2.96594202e+00, 2.87517387e+00,
       2.62491257e+00, 3.02561873e+00, 3.64461156e+00, 3.43818411e+00,
       2.74372311e+00, 3.22053971e+00, 3.08753021e+00, 4.08175446e+00,
       1.50086064e+00, 2.00836636e+00, 4.28532952e-02, 1.66863114e-01,
       2.26378763e+00, 3.48690854e+00, 3.79945448e+00, 2.93585355e+00,
       2.89553922e+00, 4.34966105e+00, 7.04929262e-02, 4.77851809e+00,
       2.92707662e+00, 4.04026732e+00, 1.81031750e+00, 1.09658870e+00,
      

In [75]:
xgb_grid = grid(xgbr, params_xgbr)
xgb_grid

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


KeyboardInterrupt: 

In [74]:
gb_grid = grid(gb, params_gb)
gb_grid

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


ValueError: Invalid parameter gamma for estimator GradientBoostingRegressor(). Check the list of available parameters with `estimator.get_params().keys()`.